<a href="https://colab.research.google.com/github/rohbot25/Erohan_Repository/blob/main/diffLudobots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install taichi==1.1.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00


In [ ]:
#simulates it
import taichi as ti
import numpy as np
import os
import math

max_steps = 50

dt = 0.01

learning_rate = 1.0

gravity = -9.8
damping = 0.1

stiffness = 1000.0

ground_height = 0.1

startingObjectPositions = []

#n_objects = 4

#for objectIndex in range(n_objects):

#  startingObjectPositions.append( [np.random.random(), np.random.random()* 0.9 + 0.1] )

startingObjectPositions.append( [0.1,ground_height+0.1])
startingObjectPositions.append( [0.1,ground_height+0.2])

startingObjectPositions.append( [0.2,ground_height+0.1])
startingObjectPositions.append( [0.2,ground_height+0.2])

n_objects = len(startingObjectPositions)

springs = []

def add_spring(objA,objB,motorized):
  objAIndex = objA
  objAPos = startingObjectPositions[objAIndex]
  objBIndex = objB
  objBPos = startingObjectPositions[objBIndex]
  xDistAB = objAPos[0] - objBPos[0]
  yDistAB = objAPos[1] - objBPos[1]
  distAB = math.sqrt( xDistAB**2 + yDistAB**2 )
  restingLength = distAB
  springs.append( [objAIndex, objBIndex, restingLength,motorized] )

add_spring(0,1,1)
add_spring(1,2,1)
add_spring(2,3,1)
add_spring(3,0,1)
add_spring(0,2,1)
add_spring(1,3,1)

n_springs = len ( springs )

real = ti.f32

ti.init( default_fp = real )

spring_anchor_a = ti.field(ti.i32)
spring_anchor_b = ti.field(ti.i32)
spring_resting_lengths = ti.field(ti.f32)
spring_actuation = ti.field(ti.i32)

vec = lambda: ti.Vector.field(2, dtype = real)

positions = vec()

velocities = vec()

springRestoringForces = vec()
springForcesOnObjects = vec()

weightsSH = ti.field(ti.f32)

n_sin_waves = 10

n_hidden_neurons = 32

center = vec()

goal = vec()

hidden = ti.field(ti.f32)

biasH = ti.field(ti.f32)

def n_sensors():
  return n_sin_waves + 4 * n_objects

weightsHM = ti.field(ti.f32)

actuation = ti.field(ti.f32)

ti.root.dense(ti.i, max_steps).dense(ti.j, n_objects).place(positions)
ti.root.dense(ti.i, max_steps).dense(ti.j, n_objects).place(positions.grad)
ti.root.dense(ti.i, max_steps).dense(ti.j, n_objects).place(velocities)
ti.root.dense(ti.i, n_springs).place(spring_anchor_a,spring_anchor_b,spring_resting_lengths,spring_actuation)
ti.root.dense(ti.i, max_steps).dense(ti.j, n_springs).place(springRestoringForces)
ti.root.dense(ti.i, max_steps).dense(ti.j, n_objects).place(springForcesOnObjects)
ti.root.dense(ti.ij, (n_hidden_neurons, n_sensors())).place(weightsSH)
ti.root.dense(ti.i, max_steps).place(center)
ti.root.place(goal)
ti.root.dense(ti.ij, (max_steps,n_hidden_neurons)).place(hidden)
ti.root.dense(ti.i, n_hidden_neurons).place(biasH)
ti.root.dense(ti.ij, (n_springs, n_hidden_neurons)).place(weightsHM)
ti.root.dense(ti.ij, (max_steps,n_springs)).place(actuation)

loss = ti.field( dtype=ti.f32, shape = ())

ti.root.lazy_grad()

@ti.kernel
def Compute_Loss():
  loss[None] -= positions[max_steps-1,0][0]

@ti.kernel
def Calculate_Center(timeStep: ti.i32):
  for _ in range(1):
    c = ti.Vector( [0.0, 0.0])

    for i in range( n_objects):

      c+= positions[timeStep, i]

    center[timeStep] = c / n_objects
# ------------------------------------------

def Draw(frameOffset):
  for timeStep in range(0,max_steps):

    gui = ti.GUI("Robot" , (512,512), background_color = 0xFFFFFF, show_gui=False)

    #draw the floor
    gui.line( begin = ( 0, ground_height) ,
              end = (1,ground_height) ,
              color = 0x0,
              radius = 3)
    #draw objects
    for objectIndex in range(n_objects):

      x = positions[timeStep,objectIndex][0]
      y = positions[timeStep,objectIndex][1]

      gui.circle ( (x,y), color = 0x0 , radius = 7 )

    #draw the springs
    for springIndex in range(n_springs):
      objectAIndex = spring_anchor_a[springIndex]
      objectBIndex = spring_anchor_b[springIndex]
      posA = positions[timeStep,objectAIndex]
      posB = positions[timeStep,objectBIndex]
      motorInside = springs[springIndex][3]
      if motorInside:
        gui.line( begin = posA, end = posB , color = 0x0, radius = 3)
      else:
        gui.line( begin = posA, end = posB , color = 0x0, radius = 1)
    gui.show( 'test' + str(frameOffset + timeStep) + '.png')

# ----------------------------------------
def Initialize():

  for objectIndex in range(n_objects):

    positions[0,objectIndex] = startingObjectPositions[objectIndex]

    velocities[0,objectIndex] = [0,-0.1]

  for springIndex in range(n_springs):
    s = springs[springIndex]
    spring_anchor_a[springIndex]        = s[0]
    spring_anchor_b[springIndex]        = s[1]
    spring_resting_lengths[springIndex] = s[2]
    spring_actuation[springIndex]       = s[3]

  for i in range(1,max_steps):
    for j in range(n_objects):
      positions[i,j][0] = 0.0
      positions[i,j][1] = 0.0
      velocities[i,j][0] = 0.0
      velocities[i,j][1] = 0.0

  for i in range(max_steps):
    for j in range(n_springs):
      springRestoringForces[i,j][0] = 0.0
      springRestoringForces[i,j][1] = 0.0

  for i in range(max_steps):
    for j in range(n_objects):
      springForcesOnObjects[i,j][0] = 0.0
      springForcesOnObjects[i,j][1] = 0.0

  for i in range(max_steps):
    for j in range(n_hidden_neurons):
      hidden[i,j] = 0.0

  for i in range(max_steps):
    for j in range(n_springs):
      actuation[i,j] = 0.0

  goal[None] = [0.9 , 0.2]

def Initialize_Neural_Network_Parameters():

  for i in range(n_hidden_neurons):
    for j in range( n_sensors()):
      weightsSH[i,j] = np.random.randn() * 0.2 - 0.1

  for i in range(n_hidden_neurons):
    biasH[i] = np.random.randn() * 2 - 1


  for i in range(n_springs):
    for j in range( n_hidden_neurons):
      weightsHM[i,j] = np.random.randn() * 0.2 - 0.1
# ----------------------------------------
def Simulate():

  for timeStep in range(1,max_steps):

    Step_One(timeStep)

# ----------------------------------------
@ti.kernel
def Simulate_Springs(timeStep : ti.i32):

  for springIndex in range(n_springs):
    objectAIndex = spring_anchor_a[springIndex]
    objectBIndex = spring_anchor_b[springIndex]
    posA = positions[timeStep-1,objectAIndex]
    posB = positions[timeStep-1,objectBIndex]
    dist = posA - posB
    length = dist.norm()
    springRestingLength = spring_resting_lengths[springIndex]

    springRestingLength = springRestingLength + \
    0.02 * spring_actuation[springIndex] * \
    actuation[timeStep,springIndex]
    #ti.sin(0.5*timeStep)

    springUnhappiness = length - springRestingLength
    springRestoringForces[timeStep,springIndex] = \
    (dt * springUnhappiness * stiffness / length) * dist

    springForcesOnObjects[timeStep,objectAIndex] += -5.0*springRestoringForces[timeStep,springIndex]
    springForcesOnObjects[timeStep,objectBIndex] += 5.0*springRestoringForces[timeStep,springIndex]

@ti.kernel
def Simulate_Objects(timeStep: ti.i32):

  for objectIndex in range(n_objects):

    oldPosition = positions[timeStep-1, objectIndex]

    oldVelocity = (1-damping) * velocities[timeStep-1, objectIndex] + \
    dt * gravity * ti.Vector([0,1]) + \
    springForcesOnObjects[timeStep,objectIndex]

    if (oldPosition[1] <= ground_height) and (oldVelocity[1] < 0):

      oldVelocity = ti.Vector([0,0])

    newPosition = oldPosition + dt * oldVelocity

    positions[timeStep,objectIndex] = newPosition

    newVelocity = oldVelocity

    velocities[timeStep,objectIndex] = newVelocity

@ti.kernel
def Simulate_Neural_Network_HM(timeStep : ti.i32):

  for i in range(n_springs):

    activation = 0.0

    for j in ti.static(range( n_hidden_neurons)):

      activation += weightsHM[i,j] * hidden[timeStep,j]

    activation = ti.tanh(activation)

    actuation[timeStep,i] = activation


@ti.kernel
def Simulate_Neural_Network_SH(timeStep : ti.i32):

  for i in range( n_hidden_neurons):

      activation = 0.0
      for j in ti.static(range( n_sin_waves)):

        activation += weightsSH[i,j] * ti.sin(10 *  timeStep * dt + \
                                              2 * math.pi / n_sin_waves * j)
      for j in ti.static(range( n_objects )):
        offset = positions[timeStep,j] - center[timeStep]

        activation += 0.1 *  weightsSH[i,j *4     + n_sin_waves] * offset[0]
        activation += 0.1 *  weightsSH[i,j +4 + 1 + n_sin_waves] * offset[1]

        activation += 0.1 *  weightsSH[i,j *4 + 2 + n_sin_waves] * positions[timeStep,j][0]
        activation += 0.1 *  weightsSH[i,j +4 + 3 + n_sin_waves] * positions[timeStep,j][1]

      activation += 0.1 * weightsSH[i,n_objects * 4 + n_sin_waves] * \
      (goal[None][0] - center[timeStep][0])

      activation += 0.1 * weightsSH[i,n_objects * 4 + n_sin_waves + 1] * \
      (goal[None][1] - center[timeStep][1])

      activation += biasH[i]

      activation = ti.tanh(activation )

      hidden[timeStep,i] = activation

def Step_One(timeStep: ti.i32):

  Calculate_Center(timeStep)

  Simulate_Neural_Network_SH(timeStep)

  Simulate_Neural_Network_HM(timeStep)

  Simulate_Springs(timeStep)

  Simulate_Objects(timeStep)

# ----------------------------------------

def Make_Movie():

  os.system("rm movie.mp4")
  os.system(" ffmpeg -i test%d.png movie.mp4")

# ----------------------------------------
def Run_Simulation():
  Initialize()

  with ti.ad.Tape(loss):

    Simulate()

    loss[None] = 0
    Compute_Loss()

  print(loss[None])



def Tune_HM_Weights():

  for i in range(n_springs):
    for j in range(n_hidden_neurons):
      weightsHM[i,j] -= learning_rate * weightsHM.grad[i,j]

def Tune_Hidden_Layer_Biases():
  for i in range(n_hidden_neurons):
    biasH[i] -= learning_rate * biasH.grad[i]

def Tune_SH_Weights():
  #tune the weights of the synapses that connect the sensors
  #to the hidden neurons
  for i in range( n_hidden_neurons):

      activation = 0.0
      for j in ti.static(range( n_sin_waves)):

        weightsSH[i,j] -= learning_rate * weightsSH.grad[i,j]

      for j in ti.static(range( n_objects )):

        weightsSH[i,j *4     + n_sin_waves] -= learning_rate * weightsSH.grad[i,j *4     + n_sin_waves]
        weightsSH[i,j +4 + 1 + n_sin_waves] -= learning_rate * weightsSH.grad[i,j +4 + 1 + n_sin_waves]

        weightsSH[i,j *4 + 2 + n_sin_waves] -= learning_rate * weightsSH.grad[i,j *4 + 2 + n_sin_waves]
        weightsSH[i,j +4 + 3 + n_sin_waves] -= learning_rate * weightsSH.grad[i,j +4 + 3 + n_sin_waves]

      weightsSH[i,n_objects * 4 + n_sin_waves] -= learning_rate * weightsSH.grad[i,n_objects * 4 + n_sin_waves]

      weightsSH[i,n_objects * 4 + n_sin_waves + 1] -= learning_rate * weightsSH.grad[i,n_objects * 4 + n_sin_waves + 1]

def Tune_Robots_Brain():

  Tune_HM_Weights()

  Tune_Hidden_Layer_Biases()

  Tune_SH_Weights()


# ---------- Main body of code

Initialize_Neural_Network_Parameters()

for optimizationStep in range(2):

  Run_Simulation()
  if(optimizationStep == 0):
    os.system("rm *.png")
    Draw(0)

  Tune_Robots_Brain()

Draw(max_steps)

Make_Movie()

#watch movie
from IPython.display import HTML
from base64 import b64encode
mp4 = open('movie.mp4', 'rb').read()
data_url = "data:video/mp4;base64,"+ b64encode(mp4).decode()

HTML('<video width=512 controls> <source src="%s" type="video/mp4"></video>' % data_url)



[Taichi] version 1.1.2, llvm 10.0.0, commit f25cf4a2, linux, python 3.10.12
[Taichi] Starting on arch=x64
-0.09373027086257935
-0.09413482993841171
